In [1]:
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
import pandas as pd
from pathlib import Path
from daily_price import load_daily_prices

pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [2]:
load_dotenv()

PGHOST = os.getenv("PGHOST")
PGPORT = os.getenv("PGPORT", "5432")
PGDATABASE = "mse_lab"
PGUSER = os.getenv("PGUSER")
PGPASSWORD = os.getenv("PGPASSWORD")

In [3]:
engine = create_engine(
    f"postgresql+psycopg2://{PGUSER}:{PGPASSWORD}@{PGHOST}:{PGPORT}",
    pool_pre_ping=True,
)

In [ ]:
with engine.connect() as conn:
    conn.execute(text("COMMIT"))
    conn.execute(text("CREATE DATABASE mse_lab"))


ProgrammingError: (psycopg2.errors.DuplicateDatabase) database "mse_lab" already exists

[SQL: CREATE DATABASE mse_lab]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [5]:
# with engine.connect() as conn:
#     who = conn.execute(text("SELECT current_user, current_database();")).fetchone()
#     print("Connected as:", who)

In [ ]:
# with engine.connect() as conn:
#     conn.execute(text("DROP TABLE counters;"))
#     conn.commit()

In [ ]:
# with engine.connect() as conn:
#     conn.execute(text("DROP TABLE prices_daily;"))
#     conn.commit()

In [36]:
engine = create_engine(
    f"postgresql+psycopg2://{PGUSER}:{PGPASSWORD}@{PGHOST}:{PGPORT}/{PGDATABASE}",
    pool_pre_ping=True,
)

In [37]:
with engine.connect() as conn:
    conn.execute(
        text(
            """
    CREATE TABLE counters (
        counter_id SERIAL PRIMARY KEY,
        ticker TEXT,
        name TEXT,
        date_listed DATE,
        listing_price NUMERIC
    )
    """
        )
    )
    conn.commit()

In [38]:
with engine.connect() as conn:
    conn.execute(
        text(
            """
            CREATE TABLE prices_daily (
                counter_id INT REFERENCES counters(counter_id),
                trade_date DATE,
                open_mwk NUMERIC,
                high_mwk NUMERIC,
                low_mwk NUMERIC,
                close_mwk NUMERIC,
                volume NUMERIC
            )
            """
        )
    )
    conn.commit()

# READ DATA

In [39]:
DIR_DATA = Path.cwd().parents[1] / "data"
DIR_MSE = DIR_DATA / "mse_main.csv"
DIR_DAILY = DIR_DATA / "mse-daily-data"

In [40]:
df_main = pd.read_csv(DIR_MSE)
df_main.head()

,name,ticker,listing_price,date_listed
0,AIRTEL MALAWI PLC,AIRTEL,12.690,24 February 2020
1,BLANTYRE HOTELS PLC,BHL,0.840,25 March 1997
2,FDH BANK PLC,FDHB,10.000,03 August 2020
3,FMB CAPITAL HOLDINGS PLC,FMBCH,45.010,18 September 2017
4,ICON PROPERTIES PLC,ICON,8.750,21 January 2019


In [41]:
# WRITE DATA into SQL
df_main.to_sql(
    "counters",  
    engine,  
    if_exists="append",  # append data to existing table
    index=False, 
)

16

In [42]:
df_counter = pd.read_sql("SELECT * FROM counters;", engine)
df_counter

,counter_id,ticker,name,date_listed,listing_price
0,1,AIRTEL,AIRTEL MALAWI PLC,2020-02-24,12.690
1,2,BHL,BLANTYRE HOTELS PLC,1997-03-25,0.840
2,3,FDHB,FDH BANK PLC,2020-08-03,10.000
3,4,FMBCH,FMB CAPITAL HOLDINGS PLC,2017-09-18,45.010
4,5,ICON,ICON PROPERTIES PLC,2019-01-21,8.750
5,6,ILLOVO,ILLOVO SUGAR MALAWI PLC,1997-11-10,2.250
6,7,MPICO,MPICO PLC,2007-11-12,2.250
7,8,NBM,NATIONAL BANK OF MALAWI,2000-08-21,4.000
8,9,NBS,NBS BANK PLC,2007-06-25,2.600
9,10,NICO,NICO HOLDINGS PLC,1996-11-11,2.000


In [43]:
load_daily = load_daily_prices(
    engine.url,
    DIR_DAILY,
)
load_daily

Loading mse-daily-2021-01-11.csv...
 Inserted 16 rows from mse-daily-2021-01-11.csv
Loading mse-daily-2025-08-20.csv...
 Inserted 16 rows from mse-daily-2025-08-20.csv
Loading mse-daily-2021-09-14.csv...
 Inserted 16 rows from mse-daily-2021-09-14.csv
Loading mse-daily-2021-11-01.csv...
 Inserted 16 rows from mse-daily-2021-11-01.csv
Loading mse-daily-2025-06-06.csv...
 Inserted 16 rows from mse-daily-2025-06-06.csv
Loading mse-daily-2024-01-16.csv...
 Inserted 16 rows from mse-daily-2024-01-16.csv
Loading mse-daily-2023-09-22.csv...
 Inserted 16 rows from mse-daily-2023-09-22.csv
Loading mse-daily-2025-01-08.csv...
 Inserted 16 rows from mse-daily-2025-01-08.csv
Loading mse-daily-2022-04-25.csv...
 Inserted 16 rows from mse-daily-2022-04-25.csv
Loading mse-daily-2025-09-01.csv...
 Inserted 16 rows from mse-daily-2025-09-01.csv
Loading mse-daily-2024-05-31.csv...
 Inserted 16 rows from mse-daily-2024-05-31.csv
Loading mse-daily-2022-03-01.csv...
 Inserted 16 rows from mse-daily-2022-03

In [44]:
df_daily = pd.read_sql("SELECT * FROM prices_daily;", engine)
df_daily.head()

,counter_id,trade_date,open_mwk,high_mwk,low_mwk,close_mwk,volume
0,1,2021-01-11,0.000,27.980,27.980,27.980,1392.000
1,2,2021-01-11,0.000,NaN,NaN,12.930,0.000
2,3,2021-01-11,0.000,14.450,14.450,14.450,1348.000
3,4,2021-01-11,26.000,NaN,NaN,26.000,0.000
4,5,2021-01-11,10.500,NaN,NaN,12.270,0.000


## Explore the Data with Queries
#### Write SQL queries to answer the following:

###### 1. Which counters are listed on the Malawi Stock Exchange, and when were they listed?


In [52]:
query1 = """
SELECT counter_id, name, date_listed
FROM counters
ORDER BY date_listed;
"""
df1 = pd.read_sql(query1, engine)
df1

,counter_id,name,date_listed
0,10,NICO HOLDINGS PLC,1996-11-11
1,2,BLANTYRE HOTELS PLC,1997-03-25
2,6,ILLOVO SUGAR MALAWI PLC,1997-11-10
3,14,STANDARD BANK MALAWI PLC,1998-06-29
4,8,NATIONAL BANK OF MALAWI,2000-08-21
5,15,SUNBIRD TOURISM PLC,2002-08-21
6,11,NATIONAL INVESTMENT TRUST PLC,2005-03-21
7,9,NBS BANK PLC,2007-06-25
8,7,MPICO PLC,2007-11-12
9,16,TELEKOM NETWORKS MALAWI PLC,2008-11-03


In [46]:
with engine.connect() as conn:
    df_cols = pd.read_sql("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'counters';", conn)
print(df_cols)


     column_name data_type
0     counter_id   integer
1         ticker      text
2           name      text
3    date_listed      date
4  listing_price   numeric


###### 2. What was the closing price of each stock on 2025-09-09?


In [53]:
query2 = """
SELECT c.name, p.trade_date, p.close_mwk
FROM prices_daily p
JOIN counters c ON p.counter_id = c.counter_id
WHERE p.trade_date = '2025-09-09'
ORDER BY c.name;
"""
df2 = pd.read_sql(query2, engine)
df2

,name,trade_date,close_mwk
0,AIRTEL MALAWI PLC,2025-09-09,138.820
1,BLANTYRE HOTELS PLC,2025-09-09,15.010
2,FDH BANK PLC,2025-09-09,607.340
3,FMB CAPITAL HOLDINGS PLC,2025-09-09,1677.070
4,ICON PROPERTIES PLC,2025-09-09,17.950
5,ILLOVO SUGAR MALAWI PLC,2025-09-09,1791.430
6,MPICO PLC,2025-09-09,19.510
7,NATIONAL BANK OF MALAWI,2025-09-09,9426.300
8,NATIONAL INVESTMENT TRUST PLC,2025-09-09,1672.100
9,NBS BANK PLC,2025-09-09,1024.820


###### 3. Which stock had the highest total trading volume in 2023?


In [54]:
query3 = """
SELECT c.name, SUM(p.volume) AS total_volume
FROM prices_daily p
JOIN counters c ON p.counter_id = c.counter_id
WHERE EXTRACT(YEAR FROM p.trade_date) = 2023
GROUP BY c.name
ORDER BY total_volume DESC
LIMIT 1;
"""
df3 = pd.read_sql(query3, engine)
df3

,name,total_volume
0,FDH BANK PLC,145891011.000


###### 4. How did the closing price of NBM change during 2024?


In [56]:
query4 = """
SELECT p.trade_date, p.close_mwk
FROM prices_daily p
JOIN counters c ON p.counter_id = c.counter_id
WHERE c.ticker = 'NBM'
  AND EXTRACT(YEAR FROM p.trade_date) = 2024
ORDER BY p.trade_date;
"""
df4 = pd.read_sql(query4, engine)
df4

,trade_date,close_mwk
0,2024-01-02,2101.260
1,2024-01-03,2101.350
2,2024-01-04,2101.350
3,2024-01-05,2101.350
4,2024-01-08,2101.370
...,...,...
230,2024-12-23,3462.130
231,2024-12-24,3462.140
232,2024-12-27,3462.140
233,2024-12-30,3462.140


###### 5. Compare the listing price vs. most recent closing price for each counter.


In [50]:
query5 = """
SELECT 
    c.name, 
    c.date_listed, 
    p.close_mwk AS listing_price, 
    l.latest_close, 
    l.latest_trade_date
FROM counters c
JOIN prices_daily p 
    ON c.counter_id = p.counter_id 
   AND p.trade_date = c.date_listed
JOIN (
    SELECT counter_id, close_mwk AS latest_close, trade_date AS latest_trade_date
    FROM prices_daily p1
    WHERE trade_date = (
        SELECT MAX(p2.trade_date)
        FROM prices_daily p2
        WHERE p2.counter_id = p1.counter_id
    )
) l 
    ON c.counter_id = l.counter_id
ORDER BY c.name;

"""

In [51]:
df1 = pd.read_sql(query1, engine)
df1

,counter_id,name,date_listed
0,10,NICO HOLDINGS PLC,1996-11-11
1,2,BLANTYRE HOTELS PLC,1997-03-25
2,6,ILLOVO SUGAR MALAWI PLC,1997-11-10
3,14,STANDARD BANK MALAWI PLC,1998-06-29
4,8,NATIONAL BANK OF MALAWI,2000-08-21
5,15,SUNBIRD TOURISM PLC,2002-08-21
6,11,NATIONAL INVESTMENT TRUST PLC,2005-03-21
7,9,NBS BANK PLC,2007-06-25
8,7,MPICO PLC,2007-11-12
9,16,TELEKOM NETWORKS MALAWI PLC,2008-11-03


In [31]:
with engine.connect() as conn:
    df1 = pd.read_sql(query1, conn)
    df2 = pd.read_sql(query2, conn)
    df3 = pd.read_sql(query3, conn)
    df4 = pd.read_sql(query4, conn)
    df5 = pd.read_sql(query5, conn)

In [29]:
print("Counters listed:")
print(df1)

print(" Closing prices on 2025-09-09:")
print(df2)

print("Stock with highest volume in 2023:")
print(df3)

print(" NBM closing prices in 2024:")
print(df4)

print(" Listing vs most recent closing price:")
print(df5)

Counters listed:
    counter_id                           name date_listed
0           10              NICO HOLDINGS PLC  1996-11-11
1           26              NICO HOLDINGS PLC  1996-11-11
2           42              NICO HOLDINGS PLC  1996-11-11
3            2            BLANTYRE HOTELS PLC  1997-03-25
4           34            BLANTYRE HOTELS PLC  1997-03-25
5           18            BLANTYRE HOTELS PLC  1997-03-25
6           22        ILLOVO SUGAR MALAWI PLC  1997-11-10
7            6        ILLOVO SUGAR MALAWI PLC  1997-11-10
8           38        ILLOVO SUGAR MALAWI PLC  1997-11-10
9           30       STANDARD BANK MALAWI PLC  1998-06-29
10          46       STANDARD BANK MALAWI PLC  1998-06-29
11          14       STANDARD BANK MALAWI PLC  1998-06-29
12          24        NATIONAL BANK OF MALAWI  2000-08-21
13          40        NATIONAL BANK OF MALAWI  2000-08-21
14           8        NATIONAL BANK OF MALAWI  2000-08-21
15          31            SUNBIRD TOURISM PLC  2002-08-

In [32]:
df5

,name,date_listed,listing_price,latest_close,latest_trade_date
